In [1]:
# 初回のインストール
using Pkg
Pkg.add("CxxCall")

    Updating registry at `~/.julia/registries/General.toml`


   Resolving package versions...


   Installed CxxCall ─ v0.2.0


    Updating `~/.julia/environments/v1.8/Project.toml`
  [dbaf2c2e] + CxxCall v0.2.0
    Updating `~/.julia/environments/v1.8/Manifest.toml`
  [dbaf2c2e] + CxxCall v0.2.0


Precompiling project...


  ✓ CxxCall
  1 dependency successfully precompiled in 1 seconds. 161 already precompiled. 2 skipped during auto due to previous errors.


In [1]:
using CxxCall

### Hello World

In [2]:
# コンパイルのための情報
dir = mktempdir()
lib = joinpath(dir, "libHelloWorld")
filepath = joinpath(dir, "HelloWorld.cxx")
eval(cxxsetup())
eval(cxxnewfile(filepath,
"""
#include <iostream>
"""))

@cxx lib function hello(name::Cstring)::Cvoid
    """
    std::cout << "Hello " << name << "!" << std::endl;
    """
end

hello (generic function with 1 method)

In [3]:
# コンパイル
using Libdl
cxx_write_code!()
libpath = "$(lib).$dlext"
run(`g++ -shared -fPIC $(filepath) -o $libpath`)


Generating C++ code:
Generating /var/folders/r4/j4n6nlf11mb1f4rpp3q81q0c0000gn/T/jl_itGZZj/HelloWorld.cxx...
Done.


Process(`g++ -shared -fPIC /var/folders/r4/j4n6nlf11mb1f4rpp3q81q0c0000gn/T/jl_itGZZj/HelloWorld.cxx -o /var/folders/r4/j4n6nlf11mb1f4rpp3q81q0c0000gn/T/jl_itGZZj/libHelloWorld.dylib`, ProcessExited(0))

In [4]:
dlopen(libpath)
hello("world")

Hello world!


### add

In [6]:
module Wrapper
    using CxxCall
    dir = mktempdir()
    lib = joinpath(dir, "libAddCxx")
    filepath = joinpath(dir, "AddCxx.cxx")
    eval(cxxsetup())
    eval(cxxnewfile(filepath,
    """
    #include <iostream>
    """))
    
    @cxx lib function add(x::Cint, y::Cint)::Cfloat
        """
        float ret = x + y;
        std::cout << "Welcome to libAddCxx" << std::endl;
        std::cout << "x=" << x << " y=" << y << " ret=" << ret << std::endl;
        return ret;
        """
    end

    @cxx lib function add_sloppy(x::Convert(Any=>Cint), y::Convert(Int64=>Cint))::Cfloat
        """
        float ret = x + y;
        return ret;
        """
    end
end#module Wrapper

Main.Wrapper

In [12]:
using Test
import .Wrapper
using Libdl


Wrapper.cxx_write_code!()
libpath = "$(Wrapper.lib).$dlext"
run(`g++ -shared -fPIC $(Wrapper.filepath) -o $libpath`)
dlopen(libpath)
Wrapper.add(Cint(1),Cint(2))

Generating C++ code:
Generating /var/folders/r4/j4n6nlf11mb1f4rpp3q81q0c0000gn/T/jl_5uJxE1/AddCxx.cxx...
Done.


Welcome to libAddCxx
x=1 y=2 ret=3


3.0f0

In [13]:
# まとめてテスト  最初の!ispathでエラーが出るのが正常
@testset "AddCxx" begin
    @test !ispath(Wrapper.filepath)
    Wrapper.cxx_write_code!()
    @test isfile(Wrapper.filepath)
    libpath = "$(Wrapper.lib).$dlext"
    run(`g++ -shared -fPIC $(Wrapper.filepath) -o $libpath`)
    @test isfile(libpath)
    dlopen(libpath)
    @test Wrapper.add(Cint(1),Cint(2)) === 3f0
    @test_throws MethodError Wrapper.add(Int64(1),Cint(2)) === 3f0
    @test Wrapper.add_sloppy(Int128(1), Int64(2)) === 3f0
    @test Wrapper.add_sloppy(Float16(1), Int64(2)) === 3f0
    @test_throws MethodError Wrapper.add_sloppy(Float16(1), Int32(2)) === 3f0
end

AddCxx: Test Failed at /Users/smamitsu/Github/Julia_study/cppcall/CxxCall/hello.ipynb:3
  Expression: !(ispath(Wrapper.filepath))
Stacktrace:
 [1] macro expansion
   @ /Applications/Julia-1.8.app/Contents/Resources/julia/share/julia/stdlib/v1.8/Test/src/Test.jl:464 [inlined]
 [2] macro expansion
   @ ~/Github/Julia_study/cppcall/CxxCall/hello.ipynb:3 [inlined]
 [3] macro expansion
   @ /Applications/Julia-1.8.app/Contents/Resources/julia/share/julia/stdlib/v1.8/Test/src/Test.jl:1363 [inlined]
 [4] top-level scope
   @ ~/Github/Julia_study/cppcall/CxxCall/hello.ipynb:3
Generating C++ code:
Generating /var/folders/r4/j4n6nlf11mb1f4rpp3q81q0c0000gn/T/jl_5uJxE1/AddCxx.cxx...
Done.


Welcome to libAddCxx
x=1 y=2 ret=3
Test Summary: | Pass  Fail  Total  Time
AddCxx        |    7     1      8  0.2s


TestSetException: Some tests did not pass: 7 passed, 1 failed, 0 errored, 0 broken.